# Integración Azure ML - Power BI

## **Escenario**
Tienes un modelo desplegado en Azure ML que predice si un paciente es diabético o no (`Diabetic`) basándose en los siguientes datos:
- `PatientID`
- `Pregnancies`
- `PlasmaGlucose`
- `DiastolicBloodPressure`
- `TricepsThickness`
- `SerumInsulin`
- `BMI`
- `DiabetesPedigree`
- `Age`

El objetivo es usar Power BI para:
1. Cargar los datos de los pacientes.
2. Invocar el endpoint del modelo de Azure ML para obtener las predicciones.
3. Visualizar los resultados en un informe de Power BI.



## **Configuración en Power BI**

### 1. Cargar los Datos en Power BI
1. Abre Power BI Desktop.
2. Ve a **Home > Get Data > Text/CSV** y selecciona el archivo `patients_data.csv`.
3. Carga los datos en Power BI.

### 2. Crear una Función Personalizada para Invocar el Endpoint
1. En Power Query Editor, ve a **Home > Advanced Editor**.
2. Crea una función personalizada llamada `InvokeAzureMLEndpoint` para enviar los datos al modelo y obtener las predicciones.

#### Código de la Función:


```powerquery
let
    InvokeAzureMLEndpoint = (data as list) =>
    let
        // URL del endpoint
        endpointUrl = "<endpoint_url>",
        
        // Encabezados HTTP
        headers = [
            #"Content-Type" = "application/json",
            #"Authorization" = "Bearer <token>"
        ],
        
        // Construir el JSON
        Columns = {
            "Pregnancies",
            "PlasmaGlucose",
            "DiastolicBloodPressure",
            "TricepsThickness",
            "SerumInsulin",
            "BMI",
            "DiabetesPedigree",
            "Age"
        },
        Index = {1},
        Data = {
            data
        },
        InputData = [
            columns = Columns,
            index = Index,
            data = Data
        ],
        JsonBody = [
            input_data = InputData
        ],

        // Convertir el JSON a texto
        requestBody = Json.FromValue(JsonBody),

        //requestBody = Json.FromValue([data = data]),
        
        
        // Realizar la solicitud HTTP
        response = Web.Contents(endpointUrl, [
            Headers = headers,
            Content = requestBody
        ]),
        
        // Parsear la respuesta JSON
        jsonResponse = Json.Document(response),
        Valor = List.First(jsonResponse),
        predictions = if Valor = 1 then "Diabetic" else "Non-Diabetic"
    in
        predictions
in
    InvokeAzureMLEndpoint
```



Reemplaza `<your-endpoint-url>` y `<your-api-key>` con los valores correspondientes de tu endpoint.



### 3. Invocar el Endpoint para Cada Fila
1. En Power Query Editor, selecciona la tabla con los datos de los pacientes.
2. Agrega una nueva columna personalizada:
   - Ve a **Add Column > Custom Column**.
   - Usa la función `InvokeAzureMLEndpoint` para enviar los datos de cada fila al modelo.

#### Código para la Nueva Columna:


```powerquery
InvokeAzureMLEndpoint({
    [Pregnancies],
    [PlasmaGlucose],
    [DiastolicBloodPressure],
    [TricepsThickness],
    [SerumInsulin],
    [BMI],
    [DiabetesPedigree],
    [Age]
})
```



Esto agregará una columna con las predicciones (`Diabetic` o `Non-Diabetic`) para cada paciente.



### 4. Cargar los Datos en Power BI
1. Cierra Power Query Editor y carga los datos en Power BI.
2. Ahora tendrás una tabla con las predicciones del modelo.



## **Visualización en Power BI**

### Crear un Informe
1. Arrastra la columna `PatientID` al lienzo.
2. Arrastra la columna de predicciones (`CustomColumn`) al lienzo.
3. Usa un gráfico de barras o un gráfico circular para visualizar la distribución de pacientes diabéticos y no diabéticos.

### Filtrar por Resultados
1. Agrega un filtro para mostrar solo los pacientes diabéticos o no diabéticos.
2. Usa slicers para explorar los datos según otras columnas, como `Age` o `BMI`.



## **Automatización**

Para automatizar el proceso de actualización de las predicciones, puedes configurar un flujo de trabajo en Power BI Service:

1. Publica el informe en Power BI Service.
2. Configura un **scheduled refresh** para actualizar los datos automáticamente.
3. Asegúrate de que el endpoint de Azure ML esté disponible y funcione correctamente.

